## 2.1. Fazer uma consulta no repositorio da BaseDosDados no Bigquery
### E gravar os resultados agregados numa outra tabela do BibQuery (no seu Sandbox)

#### Autenticação

In [3]:
from google.colab import auth
# Autenticação na GCP
auth.authenticate_user()
print('Authenticated')

Authenticated


#### Criando uma query de agragação

In [7]:
limit = "limit 10000"
municipio_mes_vacina = f"""SELECT 
    case 
        when data_aplicacao_vacina > current_date() then DATE_TRUNC(current_date(), month)
        else
            case 
                when data_aplicacao_vacina > DATE '2021-01-17' then DATE_TRUNC(data_aplicacao_vacina, month)
                else DATE '2021-01-01' 
            end
        end	as mes,
    id_municipio_estabelecimento as codibge_municipio
    ,codigo_vacina
    ,sum(case when left(dose_vacina,1)='1' then 1 else 0 end) as qtvac1
    ,sum(case when left(dose_vacina,1)='2' then 1 else 0 end) as qtvac2
    ,sum(case when left(dose_vacina,1)='R' then 1 else 0 end) as qtvacReforco
    ,sum(case when left(right(Trim(dose_vacina),7),1)='i' Then 1 else 0 end) as qtvacAdicional
    ,sum(case when codigo_vacina='88' then 1 else 0 end) as qtvacunica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados` 
group by id_municipio_estabelecimento, mes, codigo_vacina {limit};"""

#### Executando a query no Bigquery

In [27]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'idp-mba' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq(municipio_mes_vacina, project_id=project_id)

df.head()

,mes,codibge_municipio,codigo_vacina,qtvac1,qtvac2,qtvacReforco,qtvacAdicional,qtvacunica
0,2021-12-01,5103106,87,64,228,237,40,0
1,2021-12-01,5100359,87,21,191,14,0,0
2,2021-12-01,1200609,87,137,327,273,22,0
3,2021-12-01,1200351,86,65,90,0,0,0
4,2021-12-01,1200435,87,58,47,27,0,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   mes                10000 non-null  datetime64[ns]
 1   codibge_municipio  10000 non-null  object        
 2   codigo_vacina      10000 non-null  object        
 3   qtvac1             10000 non-null  int64         
 4   qtvac2             10000 non-null  int64         
 5   qtvacReforco       10000 non-null  int64         
 6   qtvacAdicional     10000 non-null  int64         
 7   qtvacunica         10000 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 625.1+ KB


#### Carregar os dados agregados no BigQuery

In [6]:
dataset = 'competenciasgestao'
table = 'municipio_mes_vacina'
project_id = 'idp-mba'
df.to_gbq(f"{dataset}.{table}",
              project_id,
              chunksize=40000,
              if_exists='replace'
              )

6it [00:24,  4.08s/it]


## Atividade 2.1.1:
### Itere com o objetivo de incorporar novos requisitos ao Dashboard. Exiba no painel uma tabela com o nome dos municípios, UF e a respectiva quantidade de Doses 1, 2, Única, Reforço e Adicional.

## 2.2 Query no Elasticsearch Público do Datasus (Dados Desagregados e Anonimizados)

In [2]:
!pip install elasticsearch==7.12.0 elasticsearch-dsl==7.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 334 kB 33.1 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 


In [3]:
from elasticsearch import Elasticsearch, helpers
from elasticsearch_dsl import Search
import pandas as pd
import requests

requests.urllib3.disable_warnings()


def extract_vacinas_mes(client, inicio, fim, operation):
    s = Search(using=client, index='').source(['vacina_dataAplicacao', 'vacina_codigo', 'vacina_numDose', 'estabelecimento_municipio_codigo']). \
        filter('range', vacina_dataAplicacao={'gte': inicio, 'lt': fim}). \
        filter('terms', vacina_numDose=['1', '2', '7', '38', '37', '9', '8']). \
        filter('terms', status=['final'])
    series_agg = None
    res = client.search(body=s.to_dict(), scroll='3m', size=1000)
    old_scroll_id = res['_scroll_id']
    while len(res['hits']['hits']):
        print(len(res['hits']['hits']))
        df = pd.DataFrame([r['_source'] for r in res['hits']['hits']])
        df['vacina_dataAplicacao'] = pd.to_datetime(df['vacina_dataAplicacao'], infer_datetime_format=True)
        df['mes'] = df['vacina_dataAplicacao'].dt.to_period('M').dt.to_timestamp()
        series_i = df.groupby(['mes', 'estabelecimento_municipio_codigo', 'vacina_codigo', 'vacina_numDose']).size()
        if series_agg is None:
            series_agg = series_i
        else:
            series_agg = series_i.add(series_agg, fill_value=0)

        res = client.scroll(scroll_id=old_scroll_id, scroll='2m')
        old_scroll_id = res['_scroll_id']
    df_agg = series_agg.reset_index()
    print(f"uploading: inicio={inicio}, fim={fim}, operation={operation}")
    return df_agg

    
def load_vacinas(dfv, dataset, table, project_id):    
    dfv.to_gbq(f"{dataset}.{table}", project_id, chunksize=40000, if_exists=operation)


In [ ]:
dataset = 'competenciasgestao'
table = 'municipio_mes_vacina'
project_id = 'idp-mba'

es = Elasticsearch('https://imunizacao-es.saude.gov.br:443', http_auth=("imunizacao_public", "qlto5t&7r_@+#Tlstigi"), verify_certs=False)
tasks = [('2022-01-01', '2022-02-01', 'append'), ('2022-02-01', '2022-03-01', 'append'), ('2022-03-01', '2022-04-01', 'append'),
             ('2022-04-01', '2022-05-01', 'append'), ('2022-05-01', '2022-06-01', 'append'), ('2022-06-01', '2022-07-01', 'append')]

for inicio, fim, operation in tasks:
    dfv = extract_vacinas_mes(es, inicio, fim, operation)
    load_vacinas(dfv, dataset, table, project_id)

## 2.3 Implementando Novos Requisitos

### 2a Dose Atrasada
#### Devido à subnotificação, não é possível saber exatamente quantas doses estão atrasadas. O que é possível saber, é quantas temos certeza que foram aplicadas até a data limite.
#### Regras (Até Fevereiro de 2022):
#### Dose 2 não atrasada: 56 dias para Pfizer e Astrazeneca, 28 dias para Coronavac.
#### Dose de reforço não atrasada: 120 dias para 18 anos ou mais. Menores não tomavam dose de reforço até então.

In [32]:
limit = "limit 30000"
municipio_mes_vacina = f"""SELECT 
    case 
        when data_aplicacao_vacina > current_date() then DATE_TRUNC(current_date(), month)
        else
            case 
                when data_aplicacao_vacina > DATE '2021-01-17' then DATE_TRUNC(data_aplicacao_vacina, month)
                else DATE '2021-01-01' 
            end
        end	as mes,
    id_municipio_estabelecimento as codibge_municipio
    ,codigo_vacina
    ,sum(case when left(dose_vacina,1)='1' then 1 else 0 end) as qtvac1
    ,sum(case when left(dose_vacina,1)='2' then 1 else 0 end) as qtvac2
    ,sum(case when left(dose_vacina,1)='R' then 1 else 0 end) as qtvacReforco
    ,sum(case when left(right(Trim(dose_vacina),7),1)='i' Then 1 else 0 end) as qtvacAdicional
    ,sum(case when codigo_vacina='88' then 1 else 0 end) as qtvacunica
    ,sum(case when left(dose_vacina,1)='1' and (((DATE_DIFF(current_date, data_aplicacao_vacina, day) <= 56) and (codigo_vacina in ('85','89','87','99'))) OR ((DATE_DIFF(current_date, data_aplicacao_vacina, day) <= 28) and codigo_vacina='86')) then 1 else 0 end) as qtvac2_nao_atrasadas
    ,sum(case when left(dose_vacina,1)='2' and ((DATE_DIFF(current_date, data_aplicacao_vacina, day) <= 120 and idade_paciente >= 18) or idade_paciente <= 17) then 1 else 0 end) as qtvacDADR_nao_atrasadas
FROM `basedosdados.br_ms_vacinacao_covid19.microdados` 
group by id_municipio_estabelecimento, mes, codigo_vacina {limit};"""


In [33]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'idp-mba' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

dfn = pd.io.gbq.read_gbq(municipio_mes_vacina, project_id=project_id)

dfn.head()

,mes,codibge_municipio,codigo_vacina,qtvac1,qtvac2,qtvacReforco,qtvacAdicional,qtvacunica,qtvac2_nao_atrasadas,qtvacDADR_nao_atrasadas
0,2021-12-01,5102504,86,113,96,1,0,0,0,1
1,2021-12-01,5106299,87,44,262,286,0,0,0,146
2,2021-12-01,1301605,86,50,78,0,0,0,0,0
3,2021-12-01,1300508,86,19,11,0,0,0,0,0
4,2021-12-01,1302801,86,366,172,0,0,0,0,6


### Nome dos Municípios, Latitude e Longitude dos Municípios no formato (lat,long), e UF
#### Converter o [arquivo dbf](https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/27385-localidades.html?=&t=downloads) para xlsx usando o excel. Processar o arquivo xlsx com o pandas
#### Vantagem: mais rápido do que o pandas processar o arquivo dbf.
#### URL do arquivo exportado para xlsx https://github.com/alexlopespereira/idp_competenciasdegestao/raw/main/data/BR_Localidades_2010_v1.xlsx

In [34]:
df_xlsx = pd.read_excel('https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx', dtype={"CD_GEOCODM,C,20":str})
df_xlsx.head()

,"ID,N,10,0","CD_GEOCODI,C,20","TIPO,C,10","CD_GEOCODB,C,20","NM_BAIRRO,C,60","CD_GEOCODS,C,20","NM_SUBDIST,C,60","CD_GEOCODD,C,20","NM_DISTRIT,C,60","CD_GEOCODM,C,20",...,"NM_MESO,C,100","NM_UF,C,60","CD_NIVEL,C,1","CD_CATEGOR,C,5","NM_CATEGOR,C,50","NM_LOCALID,C,60","LONG,N,24,6","LAT,N,24,6","ALT,N,24,5","GMRotation,N,24,5"
0,1,110001505000001,URBANO,1.100015e+11,Redondo,11000150500,NaN,110001505,ALTA FLORESTA D'OESTE,1100015,...,LESTE RONDONIENSE,RONDÔNIA,1,5,CIDADE,ALTA FLORESTA D'OESTE,-61.999824,-11.935540,337.735719,0
1,2,110001515000001,URBANO,NaN,NaN,11000151500,NaN,110001515,FILADÉLFIA D'OESTE,1100015,...,LESTE RONDONIENSE,RONDÔNIA,2,15,VILA,FILADÉLFIA D'OESTE,-62.043898,-12.437239,215.244429,0
2,3,110001520000001,URBANO,NaN,NaN,11000152000,NaN,110001520,IZIDOLÂNDIA,1100015,...,LESTE RONDONIENSE,RONDÔNIA,2,20,VILA,IZIDOLÂNDIA,-62.175549,-12.601415,181.044807,0
3,4,110001525000001,URBANO,NaN,NaN,11000152500,NaN,110001525,NOVA GEASE D'OESTE,1100015,...,LESTE RONDONIENSE,RONDÔNIA,2,25,VILA,NOVA GEASE D'OESTE,-62.318650,-11.919792,191.576571,0
4,5,110001530000001,URBANO,NaN,NaN,11000153000,NaN,110001530,ROLIM DE MOURA DO GUAPORÉ,1100015,...,LESTE RONDONIENSE,RONDÔNIA,2,30,VILA,ROLIM DE MOURA DO GUAPORÉ,-62.276812,-13.079806,157.285277,0


In [35]:
df_xlsx.columns

Index(['ID,N,10,0', 'CD_GEOCODI,C,20', 'TIPO,C,10', 'CD_GEOCODB,C,20',
       'NM_BAIRRO,C,60', 'CD_GEOCODS,C,20', 'NM_SUBDIST,C,60',
       'CD_GEOCODD,C,20', 'NM_DISTRIT,C,60', 'CD_GEOCODM,C,20',
       'NM_MUNICIP,C,60', 'NM_MICRO,C,100', 'NM_MESO,C,100', 'NM_UF,C,60',
       'CD_NIVEL,C,1', 'CD_CATEGOR,C,5', 'NM_CATEGOR,C,50', 'NM_LOCALID,C,60',
       'LONG,N,24,6', 'LAT,N,24,6', 'ALT,N,24,5', 'GMRotation,N,24,5'],
      dtype='object')

In [36]:
dfxlsx_geo = df_xlsx[['CD_GEOCODM,C,20','NM_MUNICIP,C,60', 'NM_UF,C,60','NM_CATEGOR,C,50','LONG,N,24,6','LAT,N,24,6']].rename(columns={'CD_GEOCODM,C,20':"codibge_municipio",'NM_MUNICIP,C,60':'nome_municipio', 'NM_UF,C,60':'uf', 'NM_CATEGOR,C,50':"categoria", 'LONG,N,24,6':"long", 'LAT,N,24,6':"lat"})
dfxlsx_geo.head()

,codibge_municipio,nome_municipio,uf,categoria,long,lat
0,1100015,ALTA FLORESTA D'OESTE,RONDÔNIA,CIDADE,-61.999824,-11.935540
1,1100015,ALTA FLORESTA D'OESTE,RONDÔNIA,VILA,-62.043898,-12.437239
2,1100015,ALTA FLORESTA D'OESTE,RONDÔNIA,VILA,-62.175549,-12.601415
3,1100015,ALTA FLORESTA D'OESTE,RONDÔNIA,VILA,-62.318650,-11.919792
4,1100015,ALTA FLORESTA D'OESTE,RONDÔNIA,VILA,-62.276812,-13.079806


In [37]:
dfxlsx_geo['categoria'] = dfxlsx_geo['categoria'].str.strip()
dfxlsx_geo = dfxlsx_geo[dfxlsx_geo['categoria']=='CIDADE']
dfxlsx_geo.loc[:, 'lat_long'] = dfxlsx_geo['lat'].astype(str) + ',' + dfxlsx_geo['long'].astype(str) 
dfxlsx_geo.head()

,codibge_municipio,nome_municipio,uf,categoria,long,lat,lat_long
0,1100015,ALTA FLORESTA D'OESTE,RONDÔNIA,CIDADE,-61.999824,-11.935540,"-11.9355403047646,-61.9998238962936"
6,1100023,ARIQUEMES,RONDÔNIA,CIDADE,-63.033269,-9.908463,"-9.9084628665672,-63.0332692780484"
7,1100031,CABIXI,RONDÔNIA,CIDADE,-60.544314,-13.499763,"-13.4997634596963,-60.5443135812009"
9,1100049,CACOAL,RONDÔNIA,CIDADE,-61.442944,-11.433865,"-11.4338650286852,-61.4429442118224"
18,1100056,CEREJEIRAS,RONDÔNIA,CIDADE,-60.818426,-13.195033,"-13.1950330320399,-60.8184261646815"


In [39]:
dfm = dfn.merge(dfxlsx_geo.loc[:, ['codibge_municipio', 'nome_municipio', 'uf','lat_long']], on='codibge_municipio', how='left')
dfm.head()

,mes,codibge_municipio,codigo_vacina,qtvac1,qtvac2,qtvacReforco,qtvacAdicional,qtvacunica,qtvac2_nao_atrasadas,qtvacDADR_nao_atrasadas,nome_municipio,uf,lat_long
0,2021-12-01,5102504,86,113,96,1,0,0,0,1,CÁCERES,MATO GROSSO,"-16.0740956303637,-57.6747623072181"
1,2021-12-01,5106299,87,44,262,286,0,0,0,146,PARANAÍTA,MATO GROSSO,"-9.66965549852606,-56.4788997201888"
2,2021-12-01,1301605,86,50,78,0,0,0,0,0,FONTE BOA,AMAZONAS,"-2.51227068454779,-66.0965029898347"
3,2021-12-01,1300508,86,19,11,0,0,0,0,0,BARREIRINHA,AMAZONAS,"-2.7981498369298,-57.0677504898938"
4,2021-12-01,1302801,86,366,172,0,0,0,0,6,MARAÃ,AMAZONAS,"-1.86643687730125,-65.5742837031641"


### Nome das Vacinas

In [42]:
map_vacina_nome = {"86": 'CORONAVAC', "98": 'CORONAVAC', "85": 'ASTRAZENECA', "89": 'ASTRAZENECA', "87": 'PFIZER', "88": 'JANSSEN', "99": 'PFIZER PEDIATRICA'}
dfm['vacinaApelido'] = dfm['codigo_vacina'].map(map_vacina_nome)

In [43]:
dfm.head()

,mes,codibge_municipio,codigo_vacina,qtvac1,qtvac2,qtvacReforco,qtvacAdicional,qtvacunica,qtvac2_nao_atrasadas,qtvacDADR_nao_atrasadas,nome_municipio,uf,lat_long,vacinaApelido
0,2021-12-01,5102504,86,113,96,1,0,0,0,1,CÁCERES,MATO GROSSO,"-16.0740956303637,-57.6747623072181",CORONAVAC
1,2021-12-01,5106299,87,44,262,286,0,0,0,146,PARANAÍTA,MATO GROSSO,"-9.66965549852606,-56.4788997201888",PFIZER
2,2021-12-01,1301605,86,50,78,0,0,0,0,0,FONTE BOA,AMAZONAS,"-2.51227068454779,-66.0965029898347",CORONAVAC
3,2021-12-01,1300508,86,19,11,0,0,0,0,0,BARREIRINHA,AMAZONAS,"-2.7981498369298,-57.0677504898938",CORONAVAC
4,2021-12-01,1302801,86,366,172,0,0,0,0,6,MARAÃ,AMAZONAS,"-1.86643687730125,-65.5742837031641",CORONAVAC


In [44]:
dataset = 'competenciasgestao'
table = 'municipio_mes_vacina'
project_id = 'idp-mba'
dfm.to_gbq(f"{dataset}.{table}",
              project_id,
              chunksize=40000,
              if_exists='replace'
              )

1it [00:03,  3.92s/it]


### Atividade 2.1.1: Implemente os novos requisitos no Data Studio.
#### 1) Faça um gráfico de mapa com quantidade de Dose 1 de vacina no mapa de calor.
#### 2) Faça um gráfico temporal mostrando a evolução de Dose 1 e Dose 2.
#### 3) Adicione controles (filtros) de município e UF.


### Intercambialidade de imunizantes

In [ ]:
query_intercambialidade = """select comb.id_combinacao, comb.uf_reforco, count(*) as qtd
from (
select vv.id_paciente
,least(1,sum(case when vv.codigo_vacina = '86' and left(vv.dose_vacina,1)='R' then 1 else 0 end )) as dr_coronavac
,least(1,sum(case when vv.codigo_vacina in ('87','99') and left(vv.dose_vacina,1)='R' then 1 else 0 end )) as dr_pfizer
,least(1,sum(case when vv.codigo_vacina = '88' and left(vv.dose_vacina,1)='R' then 1 else 0 end )) as dr_janssen
,least(1,sum(case when vv.codigo_vacina in('85','89') and left(vv.dose_vacina,1)='R' then 1 else 0 end )) as dr_astrazeneca
,max(case when left(vv.dose_vacina,1)='38' then vv.sigla_uf else '' end) as uf_reforco
,CAST(least(1,sum(case when vv.codigo_vacina = '86' and left(vv.dose_vacina,1)='1' then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina = '86' and left(vv.dose_vacina,1)='R' then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina in ('87','99') and left(vv.dose_vacina,1)='1' then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina in ('87','99') and left(vv.dose_vacina,1)='R' then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina = '88' and left(vv.dose_vacina,1) in ('1', 'Ú') then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina = '88' and left(vv.dose_vacina,1)='R' then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina in('85','89') and left(vv.dose_vacina,1)='1' then 1 else 0 end )) AS string) ||
CAST(least(1,sum(case when vv.codigo_vacina in('85','89') and left(vv.dose_vacina,1)='R' then 1 else 0 end )) AS string) as id_combinacao
from `basedosdados.br_ms_vacinacao_covid19.microdados` vv 
where vv.id_paciente in (
    SELECT v.id_paciente
    from `basedosdados.br_ms_vacinacao_covid19.microdados` v 
    where left(v.dose_vacina,1) in ('1','Ú','R')
    group by v.id_paciente 
    having count(*)>1 
     limit 10000
    ) 
group by vv.id_paciente 
having ((dr_coronavac>=1) or (dr_pfizer>=1) or (dr_astrazeneca>=1) or (dr_janssen>=1)) limit 10000
) as comb
group by comb.id_combinacao, comb.uf_reforco"""

# Inseri o limit 100000 para nao gastar muito da sua quota do BigQuery.